### Install & google mount

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install wandb==0.17.2

In [ ]:
%%capture
# Installs Unsloth, Xformers (Flash Attention) and all other packages!
!pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"

# We have to check which Torch version for Xformers (2.3 -> 0.0.27)
from torch import __version__; from packaging.version import Version as V
xformers = "xformers==0.0.27" if V(__version__) < V("2.4.0") else "xformers"
!pip install --no-deps {xformers} trl peft accelerate bitsandbytes triton datasets huggingface_hub

In [ ]:
import wandb
import torch
import numpy as np
import pandas as pd
from datasets import load_dataset
from huggingface_hub import notebook_login
from transformers import TrainingArguments
from trl import SFTTrainer
from unsloth import FastLanguageModel

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.


In [ ]:
!nvidia-smi #A100 GPU

Fri Aug 30 20:33:01 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA A100-SXM4-40GB          Off | 00000000:00:04.0 Off |                    0 |
| N/A   30C    P0              44W / 400W |      5MiB / 40960MiB |      0%      Default |
|                                         |                      |             Disabled |
+-----------------------------------------+----------------------+--

### HF & wandb login

In [ ]:
notebook_login()

In [ ]:
# Log in to Weights & Biases
wandb.login()
# Initialize and configure experiment
run = wandb.init(name="Llama3.1-unsloth_Conversationl", project="CodeMind-Llama3", job_type="training", entity="your-entity")

### Model : Llama-3.1-8B-Instruct / unsloth

In [ ]:
max_seq_length = 3000 # Choose any! We auto support RoPE Scaling internally!
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/Meta-Llama-3.1-8B-Instruct",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
    # token = "hf_...", # use one if using gated models like meta-llama/Llama-2-7b-hf
)

==((====))==  Unsloth 2024.8: Fast Llama patching. Transformers = 4.44.2.
   \\   /|    GPU: NVIDIA A100-SXM4-40GB. Max memory: 39.564 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.4.0+cu121. CUDA = 8.0. CUDA Toolkit = 12.1.
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.27.post2. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/5.70G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/234 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/55.4k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/340 [00:00<?, ?B/s]

In [ ]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
    # [NEW] "unsloth" uses 30% less VRAM, fits 2x larger batch sizes!
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
    random_state = 3407,
    use_rslora = False,  # We support rank stabilized LoRA
    loftq_config = None, # And LoftQ
)

Unsloth 2024.8 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


### Dataset : LimYeri/LeetCode_Python_Solutions_Data

In [ ]:
from unsloth.chat_templates import get_chat_template

tokenizer = get_chat_template(
    tokenizer,
    chat_template = "llama-3.1",
)

def formatting_prompts_func(examples):
    convos = examples["conversations"]
    texts = [tokenizer.apply_chat_template(convo, tokenize = False, add_generation_prompt = False) for convo in convos]
    return { "text" : texts, }
pass

from datasets import load_dataset
dataset = load_dataset("LimYeri/LeetCode_Python_Solutions_Data", split = "train")

Generating train split:   0%|          | 0/15734 [00:00<?, ? examples/s]

In [ ]:
from unsloth.chat_templates import standardize_sharegpt
dataset = standardize_sharegpt(dataset)
dataset = dataset.map(formatting_prompts_func, batched = True,)

Standardizing format:   0%|          | 0/15734 [00:00<?, ? examples/s]

Map:   0%|          | 0/15734 [00:00<?, ? examples/s]

In [ ]:
dataset[5]['conversations']

[{'content': 'You are a kind coding test teacher.\n\n', 'role': 'system'},
 {'content': "I don't know how to solve LeetCode Problem 6: Zigzag Conversion. Could you provide a Python solution and explain the approach? ",
  'role': 'user'},
 {'content': '```python\ndef convert(s: str, numRows: int) -> str:\n    if numRows == 1 or numRows >= len(s):\n        return s\n\n    rows = [""] * numRows\n    curRow = 0\n    goingDown = False\n\n    for c in s:\n        rows[curRow] += c\n        if curRow == 0 or curRow == numRows - 1:\n            goingDown = not goingDown\n        curRow += 1 if goingDown else -1\n\n    return "".join(rows)\n```\n    \n    1. If the number of rows is 1 or is greater than or equal to the length of the string, the string is just returned as is.\n2. Create an array called "rows" to contain the characters that would reside in each row of the zigzag pattern.\n3. Iterate through the string one character at a time.\n4. For each character in the string, place it into th

In [ ]:
dataset[5]["text"]

'<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n\nCutting Knowledge Date: December 2023\nToday Date: 26 July 2024\n\nYou are a kind coding test teacher.\n\n<|eot_id|><|start_header_id|>user<|end_header_id|>\n\nI don\'t know how to solve LeetCode Problem 6: Zigzag Conversion. Could you provide a Python solution and explain the approach? <|eot_id|><|start_header_id|>assistant<|end_header_id|>\n\n```python\ndef convert(s: str, numRows: int) -> str:\n    if numRows == 1 or numRows >= len(s):\n        return s\n\n    rows = [""] * numRows\n    curRow = 0\n    goingDown = False\n\n    for c in s:\n        rows[curRow] += c\n        if curRow == 0 or curRow == numRows - 1:\n            goingDown = not goingDown\n        curRow += 1 if goingDown else -1\n\n    return "".join(rows)\n```\n    \n    1. If the number of rows is 1 or is greater than or equal to the length of the string, the string is just returned as is.\n2. Create an array called "rows" to contain the characters that

### Training

In [ ]:
from trl import SFTTrainer
from transformers import TrainingArguments, DataCollatorForSeq2Seq
from unsloth import is_bfloat16_supported

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    data_collator = DataCollatorForSeq2Seq(tokenizer = tokenizer),
    dataset_num_proc = 2,
    packing = False, # Can make training 5x faster for short sequences.
    args = TrainingArguments(
        per_device_train_batch_size = 8,
        gradient_accumulation_steps = 2,
        warmup_steps = 200,
        num_train_epochs = 5,
        learning_rate = 2e-4,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 20,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        report_to="wandb",
        output_dir = "/content/drive/MyDrive/Colab Notebooks/CodeMind/Llama3/outputs",
        save_strategy="epoch",
    ),
)

In [ ]:
# wandb setting
wandb.config = {
    "learning_rate": 2e-4,
    "epochs":5,
    "per_device_train_batch_size":8,
    "gradient_accumulation_steps":2,
    "optim":"adamw_8bit",
    "logging_steps":20
}
wandb.watch(model)

[]

In [ ]:
from unsloth.chat_templates import train_on_responses_only
trainer = train_on_responses_only(
    trainer,
    instruction_part = "<|start_header_id|>user<|end_header_id|>\n\n",
    response_part = "<|start_header_id|>assistant<|end_header_id|>\n\n",
)

Map:   0%|          | 0/15734 [00:00<?, ? examples/s]

In [ ]:
tokenizer.decode(trainer.train_dataset[5]["input_ids"])

'<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n\nCutting Knowledge Date: December 2023\nToday Date: 26 July 2024\n\nYou are a kind coding test teacher.\n\n<|eot_id|><|start_header_id|>user<|end_header_id|>\n\nI don\'t know how to solve LeetCode Problem 6: Zigzag Conversion. Could you provide a Python solution and explain the approach? <|eot_id|><|start_header_id|>assistant<|end_header_id|>\n\n```python\ndef convert(s: str, numRows: int) -> str:\n    if numRows == 1 or numRows >= len(s):\n        return s\n\n    rows = [""] * numRows\n    curRow = 0\n    goingDown = False\n\n    for c in s:\n        rows[curRow] += c\n        if curRow == 0 or curRow == numRows - 1:\n            goingDown = not goingDown\n        curRow += 1 if goingDown else -1\n\n    return "".join(rows)\n```\n    \n    1. If the number of rows is 1 or is greater than or equal to the length of the string, the string is just returned as is.\n2. Create an array called "rows" to contain the characters that

In [ ]:
space = tokenizer(" ", add_special_tokens = False).input_ids[0]
tokenizer.decode([space if x == -100 else x for x in trainer.train_dataset[5]["labels"]])

'                                                                       \n\n```python\ndef convert(s: str, numRows: int) -> str:\n    if numRows == 1 or numRows >= len(s):\n        return s\n\n    rows = [""] * numRows\n    curRow = 0\n    goingDown = False\n\n    for c in s:\n        rows[curRow] += c\n        if curRow == 0 or curRow == numRows - 1:\n            goingDown = not goingDown\n        curRow += 1 if goingDown else -1\n\n    return "".join(rows)\n```\n    \n    1. If the number of rows is 1 or is greater than or equal to the length of the string, the string is just returned as is.\n2. Create an array called "rows" to contain the characters that would reside in each row of the zigzag pattern.\n3. Iterate through the string one character at a time.\n4. For each character in the string, place it into the appropriate row of the "rows" array.\n5. Determine whether the next character should be placed in the row above or below the current row by checking if the current row is 0 (

In [ ]:
#@title Show current memory stats
gpu_stats = torch.cuda.get_device_properties(0)
start_gpu_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
max_memory = round(gpu_stats.total_memory / 1024 / 1024 / 1024, 3)
print(f"GPU = {gpu_stats.name}. Max memory = {max_memory} GB.")
print(f"{start_gpu_memory} GB of memory reserved.")

GPU = NVIDIA A100-SXM4-40GB. Max memory = 39.564 GB.
5.984 GB of memory reserved.


In [ ]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 15,734 | Num Epochs = 5
O^O/ \_/ \    Batch size per device = 8 | Gradient Accumulation steps = 2
\        /    Total batch size = 16 | Total steps = 4,915
 "-____-"     Number of trainable parameters = 41,943,040
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


Step,Training Loss
20,0.794300
40,0.703000
60,0.602200
80,0.559700
100,0.552500
120,0.545100
140,0.535800
160,0.520000
180,0.533500
200,0.514600


In [ ]:
wandb.finish()

In [ ]:
#@title Show final memory and time stats
used_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
used_memory_for_lora = round(used_memory - start_gpu_memory, 3)
used_percentage = round(used_memory         /max_memory*100, 3)
lora_percentage = round(used_memory_for_lora/max_memory*100, 3)
print(f"{trainer_stats.metrics['train_runtime']} seconds used for training.")
print(f"{round(trainer_stats.metrics['train_runtime']/60, 2)} minutes used for training.")
print(f"Peak reserved memory = {used_memory} GB.")
print(f"Peak reserved memory for training = {used_memory_for_lora} GB.")
print(f"Peak reserved memory % of max memory = {used_percentage} %.")
print(f"Peak reserved memory for training % of max memory = {lora_percentage} %.")

19844.616 seconds used for training.
330.74 minutes used for training.
Peak reserved memory = 23.25 GB.
Peak reserved memory for training = 17.266 GB.
Peak reserved memory % of max memory = 58.766 %.
Peak reserved memory for training % of max memory = 43.641 %.


### Save the model

In [ ]:
model.push_to_hub("LimYeri/CodeMind-Llama3.1-8B-unsloth") # Online saving
tokenizer.push_to_hub("LimYeri/CodeMind-Llama3.1-8B-unsloth") # Online saving

README.md:   0%|          | 0.00/606 [00:00<?, ?B/s]

adapter_model.safetensors:   0%|          | 0.00/168M [00:00<?, ?B/s]

Saved model to https://huggingface.co/LimYeri/CodeMind-Llama3.1-8B-unsloth


In [ ]:
# Merge to 16bit
# model.save_pretrained_merged("model", tokenizer, save_method = "merged_16bit",)
model.push_to_hub_merged("LimYeri/CodeMind-Llama3.1-8B-unsloth-merged", tokenizer, save_method = "merged_16bit")

Unsloth: You are pushing to hub, but you passed your HF username = LimYeri.
We shall truncate LimYeri/CodeMind-Llama3.1-8B-unsloth-merged to CodeMind-Llama3.1-8B-unsloth-merged
Unsloth: Kaggle/Colab has limited disk space. We need to delete the downloaded
model which will save 4-16GB of disk space, allowing you to save on Kaggle/Colab.
Unsloth: Will remove a cached repo with size 5.7G


Unsloth: Merging 4bit and LoRA weights to 16bit...
Unsloth: Will use up to 44.15 out of 83.48 RAM for saving.


100%|██████████| 32/32 [00:00<00:00, 54.70it/s]


Unsloth: Saving tokenizer... Done.
Unsloth: Saving model... This might take 5 minutes for Llama-7b...


README.md:   0%|          | 0.00/606 [00:00<?, ?B/s]

...

Upload 4 LFS files:   0%|          | 0/4 [00:00<?, ?it/s]

model-00004-of-00004.safetensors:   0%|          | 0.00/1.17G [00:00<?, ?B/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

Done.
Saved merged model to https://huggingface.co/LimYeri/CodeMind-Llama3.1-8B-unsloth-merged


In [ ]:
# Save to 16bit GGUF
# model.save_pretrained_gguf("model", tokenizer, quantization_method = "f16")
# model.push_to_hub_gguf("LimYeri/CodeMind-Llama3.1-8B-unsloth-GGUF", tokenizer, quantization_method = "f16")

### Inference

In [ ]:
max_seq_length = 3000 # Choose any! We auto support RoPE Scaling internally!
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.

In [ ]:
from unsloth import FastLanguageModel

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "LimYeri/CodeMind-Llama3.1-8B-unsloth", # YOUR MODEL YOU USED FOR TRAINING
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
)
FastLanguageModel.for_inference(model) # Enable native 2x faster inference

==((====))==  Unsloth 2024.8: Fast Llama patching. Transformers = 4.44.2.
   \\   /|    GPU: NVIDIA A100-SXM4-40GB. Max memory: 39.564 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.4.0+cu121. CUDA = 8.0. CUDA Toolkit = 12.1.
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.27.post2. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/5.70G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/234 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/55.4k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/340 [00:00<?, ?B/s]

adapter_model.safetensors:   0%|          | 0.00/168M [00:00<?, ?B/s]

Unsloth 2024.8 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): LlamaForCausalLM(
      (model): LlamaModel(
        (embed_tokens): Embedding(128256, 4096)
        (layers): ModuleList(
          (0-31): 32 x LlamaDecoderLayer(
            (self_attn): LlamaAttention(
              (q_proj): lora.Linear4bit(
                (base_layer): Linear4bit(in_features=4096, out_features=4096, bias=False)
                (lora_dropout): ModuleDict(
                  (default): Identity()
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=4096, out_features=16, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=16, out_features=4096, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
                (lora_magnitude_vector): ModuleDict()
              )
              (k_proj): lora.Linear4bit(
      

In [ ]:
from IPython.display import display, Markdown

In [ ]:
from unsloth.chat_templates import get_chat_template
from IPython.display import display, Markdown

tokenizer = get_chat_template(
    tokenizer,
    chat_template = "llama-3.1",
)
FastLanguageModel.for_inference(model) # Enable native 2x faster inference

messages = [
    {"role": "system", "content": "You are a kind coding test teacher."},
    {"role": "user", "content": "I don't know how to solve LeetCode Problem 3: Longest Substring Without Repeating Characters. Could you provide a Python solution and explain the approach?"},
]
inputs = tokenizer.apply_chat_template(
    messages,
    tokenize = True,
    add_generation_prompt = True, # Must add for generation
    return_tensors = "pt",
).to("cuda")

outputs = model.generate(input_ids = inputs, max_new_tokens = 3000, use_cache = True,
                         temperature = 1.5, min_p = 0.1)
text = tokenizer.batch_decode(outputs)

In [ ]:
text = text[0].split('assistant<|end_header_id|>\n\n')[1].strip()

In [ ]:
display(Markdown(text))

```python
class Solution:
    def lengthOfLongestSubstring(self, s: str) -> int:
        longest_length = 0
        char_set = set()
        left = 0
        for right in range(len(s)):
            while s[right] in char_set:
                char_set.remove(s[left])
                left += 1
            char_set.add(s[right])
            if len(char_set) > longest_length:
                longest_length = len(char_set)
        return longest_length
```


The provided Python code is a solution to the problem of finding the length of the longest substring without repeating characters in a given string `s`. Let's break down the code and analyze it step by step:

1. **Data Structures Used**:
   - `longest_length`: This variable stores the length of the longest substring without repeating characters found so far.
   - `char_set`: This set data structure is used to keep track of unique characters in the current substring being considered.
   - `left`: This variable represents the left pointer of the current substring being analyzed.

2. **Algorithm**:
   - The code uses a sliding window approach to find the longest substring without repeating characters.
   - It iterates through the input string `s` using a `for` loop.
   - For each character at the `right` index, it checks if that character is already present in the `char_set`.
   - If the character is already in the set, it removes characters from the left side of the substring (incrementing `left`) until the character is no longer present.
   - It then adds the current character (`s[right]`) to the set and updates the `longest_length` if the current substring's length is greater.

3. **Time Complexity**:
   - The code iterates through the input string once using a single `for` loop, resulting in a time complexity of O(n), where n is the length of the input string `s`.
   - The while loop inside the for loop can have a maximum of n iterations in total, making the overall time complexity O(n).

4. **Space Complexity**:
   - The space complexity is O(min(n, m)), where n is the length of the input string `s` and m is the size of the character set (English letters, digits, symbols, and spaces). The space complexity is determined by the size of the `char_set` and the `longest_length` variable.

5. **Example**:
   - For the input "pwwkew", the code will output 3, as the longest substring without repeating characters is "wke" with a length of 3.

In conclusion, the provided code efficiently solves the problem of finding the length of the longest substring without repeating characters using a sliding window approach with a time complexity of O(n) and a space complexity of O(min(n, m)).<|eot_id|>

In [ ]:
from unsloth.chat_templates import get_chat_template
from IPython.display import display, Markdown

tokenizer = get_chat_template(
    tokenizer,
    chat_template = "llama-3.1",
)
FastLanguageModel.for_inference(model) # Enable native 2x faster inference

messages = [
    {"role": "system", "content": "You are a kind coding test teacher."},
    {"role": "user", "content": "I don't know how to solve LeetCode Problem 121: Best Time to Buy and Sell Stock. Could you provide a Python solution and explain the approach?"},
]
inputs = tokenizer.apply_chat_template(
    messages,
    tokenize = True,
    add_generation_prompt = True, # Must add for generation
    return_tensors = "pt",
).to("cuda")

outputs = model.generate(input_ids = inputs, max_new_tokens = 3000, use_cache = True,
                         temperature = 1.5, min_p = 0.1)
text = (tokenizer.batch_decode(outputs))[0].split('assistant<|end_header_id|>\n\n')[1].strip()
display(Markdown(text))

```python
class Solution:
    def maxProfit(self, prices: List[int]) -> int:
        #Buy stock at lowest price and sell at highest price
        buy_price = float('inf')
        profit = 0
        
        for curr_price in prices:
            if(curr_price<buy_price):
                buy_price = curr_price
            else:
                profit = max(curr_price - buy_price, profit)
                
        return profit
```


The given Python code is a solution to the LeetCode problem 121 - Best Time to Buy and Sell Stock. Let's break down the code and explain it in detail:

1. **Class Definition**:
   - The code defines a class `Solution` which contains a method `maxProfit`.
   - The method `maxProfit` takes in a list of integers `prices` and returns an integer representing the maximum profit that can be achieved.

2. **Variables**:
   - `buy_price`: Represents the lowest price at which the stock can be bought. Initialized with `float('inf')` to represent infinity.
   - `profit`: Represents the maximum profit that can be achieved. Initialized to 0.

3. **Logic**:
   - The code iterates through each price in the `prices` list.
   - If the current price is lower than the `buy_price`, it updates the `buy_price` to the current price. This means that we are updating the lowest possible price to buy the stock.
   - If the current price is not lower, it calculates the profit by subtracting the `buy_price` from the current price and updates the `profit` to the maximum of the current profit and the calculated profit. This ensures that we always keep track of the maximum profit that can be achieved.

4. **Time Complexity**:
   - The code has a time complexity of O(n), where n is the number of elements in the `prices` list. This is because the code iterates through the prices list only once.

5. **Space Complexity**:
   - The code has a space complexity of O(1) as it uses a constant amount of extra space regardless of the input size. Only two extra variables (`buy_price` and `profit`) are used to keep track of the buying price and maximum profit.

6. **Explanation**:
   - The code essentially finds the maximum profit by buying the stock at the lowest price and selling it at the highest price within the given list of prices.
   - By keeping track of the lowest buying price and updating the profit accordingly, the code efficiently determines the maximum profit that can be achieved.

In conclusion, the provided code efficiently solves the problem of finding the maximum profit that can be achieved by buying and selling stocks on different days, with a time complexity of O(n) and a space complexity of O(1).<|eot_id|>

In [ ]:
from unsloth.chat_templates import get_chat_template
from IPython.display import display, Markdown

tokenizer = get_chat_template(
    tokenizer,
    chat_template = "llama-3.1",
)
FastLanguageModel.for_inference(model) # Enable native 2x faster inference

messages = [
    {"role": "system", "content": "You are a kind coding test teacher."},
    {"role": "user", "content": "I don't know how to solve LeetCode Problem 121: Best Time to Buy and Sell Stock. Could you provide a Python solution and explain the approach?"},
]
inputs = tokenizer.apply_chat_template(
    messages,
    tokenize = True,
    add_generation_prompt = True, # Must add for generation
    return_tensors = "pt",
).to("cuda")

outputs = model.generate(input_ids = inputs, max_new_tokens = 3000, use_cache = True,
                         temperature = 0.5, min_p = 0.3)
text = (tokenizer.batch_decode(outputs))[0].split('assistant<|end_header_id|>\n\n')[1].strip()
display(Markdown(text))

```python
class Solution:
    def maxProfit(self, prices: List[int]) -> int:
        max_profit = 0
        min_price = float('inf')
        for price in prices:
            min_price = min(price, min_price)
            max_profit = max(price - min_price, max_profit)
        return max_profit
```


The given code is a solution to LeetCode problem 121, which aims to find the maximum profit that can be achieved by buying and selling a stock at different days. Let's break down the code and analyze it:

### Code Explanation:
1. The `maxProfit` function takes a list of integers `prices` as input and returns an integer representing the maximum profit that can be achieved.
2. It initializes two variables `max_profit` and `min_price` to 0 and `float('inf')` respectively. `min_price` is set to positive infinity to track the minimum price encountered so far.
3. It then iterates through each price in the `prices` list.
4. For each price, it updates `min_price` to be the minimum of the current price and the previous `min_price`.
5. It calculates the profit by subtracting the `min_price` from the current price and updates `max_profit` to be the maximum of the current profit and the previous `max_profit`.
6. Finally, it returns the `max_profit` which represents the maximum profit that can be achieved.

### Data Structures Used:
- The code uses two variables `max_profit` and `min_price` to keep track of the maximum profit and minimum price encountered so far.
- It also uses a for loop to iterate through the list of prices.

### Time Complexity Analysis:
- The code iterates through the list of prices once, so the time complexity is O(n), where n is the number of prices in the input list.

### Space Complexity Analysis:
- The code uses a constant amount of extra space for the two variables `max_profit` and `min_price`, so the space complexity is O(1).

### Overall:
- The code efficiently solves the problem by keeping track of the minimum price and updating the maximum profit accordingly.
- It has a time complexity of O(n) and a space complexity of O(1), making it an optimal solution for the given problem.<|eot_id|>